# 演習III 第１回 Web スクレイピング

出典　https://code.dividable.net/tutorials/python-scraping-blog/

## 1. モジュールのインストールとインポート

In [ ]:
!pip install  --trusted-host pypi.org --trusted-host files.pythonhosted.org beautifulsoup4 japanize-matplotlib

In [ ]:
import pandas as pd
import csv
import time
import random
import re
import json

In [ ]:
from bs4 import BeautifulSoup
import requests

## 2. BeautifulSoup の初期化

BeautifulSoupで、HTMLを読み込みます。本来は、外部サイトのHTMLを取得して、そのデータを受け取りますが、学習のため、こちらで用意したHTMLを利用してみたいと思います。

In [ ]:
# こちらで用意したHTML
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""
# BeautifulSoupの初期化
soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化

In [ ]:
print(soup.prettify()) # HTMLをインデントすることができます。

## 3. BeautifulSoupでtitleを取得

BeautifulSoupを利用する方法はいたってシンプルです。

titleタグの内容を取りたい場合は、soup.titleで取得できます。

例： \<title\>The Dormouse’s story\</title\>

 titleタグの中身の文字だけを取りたい場合は、soup.title.stringで取得できます。
 
例：The Dormouse’s story

さっそく試してみましょう。

In [ ]:
# <title>The Dormouse's story</title>を取得してください。
print(soup.title)
# The Dormouse's story を取得してください。
print(soup.title.string)

## 4. BeautifulSoupで複数のaタグを取得

では、今度は同じタグが複数存在する場合も見てみましょう。現状、aタグは3つあります。これらのタグをすべて取りたいです。そこで、このaタグを取ろうとして、soup.aを試すと、最初の一つだけとることになり、すべてを取ることができません。

In [ ]:
print(soup.a)

そこで、今回は複数のタグを取る find_all メソッドを利用して、複数タグを取得します。 soup.find_all(‘a’)のように指定すると、リスト形式ですべてのaタグを取得することができます。

In [ ]:
soup.find_all("a")

それでは、取得したaタグのリンクを一つ一つ取り出してみましょう。リストの中から、一つ一つaタグを取得していきます。

## TODO
1. for文を利用して、取得したaタグのHTMLを含む部分をprintしてください。
2. for文を利用して、取得したaタグのHTMLを含まない部分だけprintしてください。
ヒント
1. リスト形式のデータは、for 単数 in リスト 構文を利用して、一つひとつ取り出すことができます。
2. HTMLを含まない中身をとるものは、stringを利用するのでした。

In [ ]:
tags = soup.find_all("a")
# for文を利用して、取得したaタグのHTMLを含む部分をprintしてください。
print("1. ")
for tag in tags:
   print (tag)
# for文を利用して、取得したaタグのHTMLを含む部分をprintしてください。
print("2. ")
for tag in tags:
   print (tag.string)

## 5. BeautifulSoupでURLの取得

今度は、取得したaタグのhrefに当たる部分、つまりURLだけとりだしてみましょう。soup.a.get(“href”)のように指定することで、URLを取得することができます。

In [ ]:
soup.a.get("href")

それでは、さきほど取得したaタグのすべてのURLをprintしてみましょう。

In [ ]:
# TODO1 for文を利用して、aタグのURLをすべてprintしてください
for link in tags:
 print (link.get("href"))

# 実際のサイトをスクレイピングしてみよう

## 1. Requests: WebページのHTMLを取得しよう
下記のページから、記事のタイトルと、その記事のURLを取得してみましょう。実際にWebページからデータを取得するのは、requestsというライブラリを利用します。 requestsをimportします。requests.get(url)でurlのページの情報を取得し、textを実行するとHTMLの内容を取得することができます。

In [ ]:
response = requests.get("https://review-of-my-life.blogspot.com/", verify='FG_CA.cer')
print (response.text)

In [ ]:
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
print(soup.prettify())

## pandas: CSVにデータを保存しよう

### TODO
- 記事名と記事URLをすべて取得し、CSVに出力してください。
- データフレームを作成してください。列名は、name, urlです。
- 記事名と記事URLをデータフレームに追加してください
- result.csvという名前でCSVに出力してください。

In [ ]:
# 記事名と記事URLの取得
tags = soup.find_all("h3",{"class":"post-title"})
print(tags)

In [ ]:
# データフレームを作成してください。列名は、name, urlです。
columns = ["name", "url"]
df = pd.DataFrame(columns=columns)
# 記事名と記事URLをデータフレームに追加してください
for tag in tags:
 name = tag.a.string
 url = tag.a.get("href")
 se = pd.Series([name, url], columns)
 print(se)
 df = df.append(se, columns)
# result.csvという名前でCSVに出力してください。
filename = "result.csv"
df.to_csv(filename, encoding = 'SHIFT-JIS', index=False) #encoding指定しないと、エラーが起こります。おまじないだともって入力します。

# 課題
売れているノートパソコンは大体いくら位のものか？スクレイピングを利用して調べよ。